In [2]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
#streaming은 model의 응답이 생성되는 것을 볼 수 있게 해준다. console에서의 응답 진행과정을 확인할 수 있음 

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat



In [3]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})


AIMessage(content="For a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use extra-firm tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marination process as you would with chicken. Tofu absorbs flavors well, so marinating it for a longer time will enhance the taste.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it gets a golden crust on the outside.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked. You can find it in Indian grocery stores or make it at home by curdling hot milk with lemon juice or vinegar.\n   - Cut the paneer into cubes and lightly fry them in a pan until the